## For this process, we will primarily use PyGrib



---
### Before running through this notebook, make sure to complete these notebooks
- [2D Variables](2D-Vars.ipynb)
- [3D Variables](3D-Vars.ipynb)
- [Surface Variables](Surface_Variables.ipynb)
---


#### Running these notebooks will create graphics to visualize the climate change deltas, in addition to netCDF files that contain the data
---



### **Import libraries and setup file paths**


#### Now that teverything is setup, let's import the neccessary libraries

In [1]:
import pygrib
import ncepgrib2
import xarray as xr
import iris
import eccodes
import cfgrib
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.units import units
import numpy as np
from scipy.ndimage import gaussian_filter
import xarray as xr
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

## Run in parallel

In [2]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
cluster = SLURMCluster()
cluster.scale(1)
client = Client(cluster)

In [5]:
client

Client Scheduler: tcp://172.22.178.4:45330 Dashboard: http://172.22.178.4:44582/status,Cluster Workers: 20 Cores: 20 Memory: 100.00 GB


---
### **Write to File**


### This function loops through grib file and edit values with climate change delta

In [9]:
def modify_grib_file(model, nam_filepath, namfile, output_file_path, setting, plot_3d=False, plot_sfc=False, plot_soil=False):
    """
    Uses climate change deltas to modify NAM analysis files
    
    Input:

    Model: String, Type of model (ex. MIROC, GFDL, or NCAR)
    
    namfile: String, Name of nam file (full filepath)
    
    output_file_path: string, file path where the output will be saved
    
    setting: String, Variables to modify with climate change deltas.
             Options: Atmos, Surface, or Soil. Use All to edit all three
    
    plot_3d: Boolean, creates 500 hPa (Height and Winds) and 700 hPa (Height and RH)
    
    plot_2d: Boolean, creates surface map (temperature and MSLP)
    
    Output:
    
    Modified name files: saved in output_file directory
    
    Figures: Creates maps
    """
    ds_2d = xr.open_dataset('../preliminary_netcdf/'+model+'_2d_vars.nc')
    ds_3d = xr.open_dataset('../preliminary_netcdf/'+model+'_3d_vars.nc')
    ds_soil = xr.open_dataset('../preliminary_netcdf/'+model+'_soil_vars.nc')
    
    # Set Lats and Lons for plotting
    lats = ds_2d.lat
    lons = ds_2d.lon
    
    # Set the file to namfile
    file = namfile
    
    # Set the time
    dt = datetime.strptime(namfile[11:-8], '%Y%m%d_%H%M')
    
    # Test this out
    atmos_vars = ['geopotential_height', 'air_temperature', 'northward_wind', 'eastward_wind']
    
    if setting == 'All':
        setting = 'Atmos, Surface, Soil'
    
    if 'Atmos' not in setting:
        atmos_vars = ['geopotential_height', 'air_temperature', 'northward_wind', 'eastward_wind', 'Relative_humidity_isobaric']
        for var in atmos_vars:
            ds_3d[var].values = ds_3d[var].values * 0
    
    if 'Surface' not in setting:
        surface_vars = ['Pressure_reduced_to_MSL_msl', 'surface_air_pressure', 'Relative humidity', '2 metre temperature',
                        '10 metre V wind component', '10 metre U wind component', 'air_temperature', 'specific_humidity']
        for var in surface_vars:
            try:
                ds_2d[var].values = ds_2d[var].values * 0
            except KeyError:
                None
            
    if 'Soil' not in setting:
        soil_vars = ['Soil Temperature', 'Soil Moisture']
        
        # Loop through the different variables and set = 0
        for var in soil_vars:
            ds_soil[var].values = ((ds_soil[var].values *0)+1)
    
    # Read in the data using pygrib
    grbs = pygrib.open(nam_filepath + file)
    
    lats = ds_2d.lat
    lons = ds_2d.lon
    
    if setting == 'Atmos, Surface, Soil':
        grbout = open(output_file_path +'All/'+ model +'_' + namfile,'wb')
    else:
        grbout = open(output_file_path +'/'+setting+'/'+ model +'_' + namfile,'wb')
    i = 0
    for grb in grbs:
    
        # Isobaric surface variables 3D
        if grb.typeOfLevel == 'isobaricInhPa':
    
            # Modify geopotential height fields
            if grb.cfName == 'geopotential_height':
                
                grb['values'] = grb.values + ds_3d['geopotential_height'].sel(isobaric = grb.level).values
                
                if grb.level == 500:
                    hght_500 = gaussian_filter(grb['values'], sigma=3.0)
                #plot_var_atmos(grb, 'Geopotential Height at ', model)
                
                elif grb.level == 700:
                    hght_700 = gaussian_filter(grb['values'], sigma=3.0)
    
            # Modify temperature fields
            elif grb.cfName == 'air_temperature':
                grb['values'] = grb.values + ds_3d['air_temperature'].sel(isobaric = grb.level).values
                #plot_var_atmos(grb, 'Temperature at ', model)
                
        
            # 3-D V-component of wind
            elif grb.cfName == 'northward_wind':
                grb['values'] = grb.values + ds_3d['northward_wind'].sel(isobaric = grb.level).values
                #plot_var_atmos(grb, 'V-Wind at', model)
                
                if grb.level == 500:
                    vwnd_500 = gaussian_filter(np.array(grb['values']), sigma=3.0) * units('m/s')
        
            # 3-D U-component of wind
            elif grb.cfName == 'eastward_wind':
                grb['values'] = grb.values + ds_3d['eastward_wind'].sel(isobaric = grb.level).values
                #plot_var_atmos(grb, 'U-Wind at ', model)
                
                if grb.level == 500:
                    uwnd_500 = gaussian_filter(np.array(grb['values']), sigma=3.0) * units('m/s')
        
            # 3-D Relative humidity
            elif grb.name == 'Relative humidity':
                rh = grb.values + ds_3d['Relative_humidity_isobaric'].sel(isobaric = grb.level).values
                
                # Check where RH > 100 and set = 100
                rh = np.where(rh < 100, rh, 100)
                rh = np.where(rh > 0, rh, 0)
                
                #print(rh.max(), rh.min())
                
                grb['values'] = rh
                
                #plot_var_atmos(grb, 'RH at', model)
                
                if grb.level == 700:
                    relh_700 = gaussian_filter(np.array(grb['values']), sigma=3.0) * units.percent

            else:
                continue

        # Variables near surface - 2D
        elif grb.typeOfLevel == 'heightAboveGround':
        
            # 2m Temperature
            if grb.name == '2 metre temperature':
                grb['values'] = np.ma.array(grb.values + ds_2d['2 metre temperature'].values, mask=False)
                
                tmpk = gaussian_filter(np.array(grb['values']), sigma=3.0) * units.kelvin
                
            elif grb.name == 'Pressure':
                grb['values'] = np.ma.array(grb.values + ds_2d['surface_air_pressure'].values, mask=False)
                
            elif grb.name == 'Temperature':
                grb['values'] = np.ma.array(grb.values + ds_2d['air_temperature'].values, mask=False)
        
            # Dewpoint
        
            # Relative humidty at 2m above surface
            elif grb.name == 'Relative humidity':
                rh = grb.values + ds_2d['Relative humidity'].values
                rh = np.where(rh < 100, rh, 100)
                rh = np.where(rh > 0, rh, 0)
                grb['values'] = np.ma.array(rh, mask=False)
            
            # U-wind at 10 meters above ground
            elif grb.name == '10 metre U wind component':
                try:
                    # Modify the surface u-component of the wind
                    grb['values'] = np.ma.array(grb.values + ds_2d['10 metre U wind component'].values, mask=False)
                except KeyError:
                    None

            # V-wind at 10 meters above ground
            elif grb.name == '10 metre V wind component':
                try:
                    # Modify the surface wind component
                    grb['values'] = np.ma.array(grb.values + ds_2d['10 metre V wind component'].values, mask=False)
                    
                except KeyError:
                    # If the surface wind component is missing (as in NCAR), use the surface winds from the NAM
                    None
            
            elif grb.name == 'Specific humidity':
                
                # Modify the specific humidity near the surface
                grb['values'] = np.ma.array(grb.values + ds_2d['specific_humidity'].values, mask=False)
            
            else:
                continue
    
        elif grb.typeOfLevel == 'surface':
            
            if grb.cfName == 'surface_air_pressure':
                grb['values'] = np.ma.array(grb.values + ds_2d['surface_air_pressure'].values, mask=False)

            # Surface temperature
            elif grb.cfName == 'air_temperature':
                grb['values'] = np.ma.array(grb.values + ds_2d['air_temperature'].values, mask=False)
                
            elif grb.name == 'Land-sea mask':
                None
                
            elif grb.name == 'Orography':
                None
            
            else:
                continue
        
        #Soil Temperature - 3D
        elif grb.name == 'Soil Temperature':
            grb['values'] = np.ma.array(grb.values * ds_soil['Soil Temperature'].sel(depth = grb.level).values, mask=False)
            soil_t = grb['values'] * units.kelvin

        # Soil moisture - 2D
        #elif grb.name == 'Soil Moisture':
        #    grb['values'] = grb.values * ds_soil['Soil Moisture'].values
    
        elif grb.indicatorOfParameter == 130:
        #elif grb.name == 'Mean sea level pressure':
            # Mean sea-level pressure
            grb['values'] = np.ma.array(grb.values + ds_2d['Pressure_reduced_to_MSL_msl'].values, mask=False)
            
            mslp = gaussian_filter(grb['values'], sigma=3.0) * units.pascal
        
        elif grb.indicatorOfParameter == 144:
            # Modify soil moisture values
            grb['values'] = np.ma.array(grb.values * ds_soil['Soil Moisture'].sel(depth=grb.level).values, mask=False)
    
        else:
            # If the message is not any of the desired variables, continue onto the next one
            continue
    
        # Write the grib message to the file
        msg = grb.tostring()
        grbout.write(msg)
    
    # Close the grib file
    grbout.close()
    
    # Create a plot of 500-hPa Height and Wind
    
    if setting == 'Control':
        model=''
    
    if plot_3d == True:
    
        # Calculate wind speed from components
        sped_500 = mpcalc.wind_speed(uwnd_500, vwnd_500).to('kt')
    
        # Set up the projection that will be used for plotting
        mapcrs = ccrs.LambertConformal(central_longitude=-100,
                                       central_latitude=35,
                                       standard_parallels=(30, 60))
    
        # Set up the projection of the data; if lat/lon then PlateCarree is what you want
        datacrs = ccrs.PlateCarree()
    
        # 500 
        # Start the figure and create plot axes with proper projection
        fig = plt.figure(1, figsize=(10, 8))
        ax = plt.subplot(111, projection=mapcrs)
        ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    
        # Add geopolitical boundaries for map reference
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
        ax.add_feature(cfeature.STATES.with_scale('50m'))
    
        # Plot 500-hPa Colorfill Wind Speeds in knots
        clevs_500_sped = np.arange(30, 100, 10)
        cf = ax.contourf(lons, lats, sped_500, clevs_500_sped, cmap=plt.cm.BuPu,
                         transform=datacrs)
        plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
    
        #Plot 500-hPa Geopotential Heights in meters
        clevs_500_hght = np.arange(0, 8000, 60)
        cs = ax.contour(lons, lats, hght_500, clevs_500_hght, colors='black',
                        transform=datacrs)
        plt.clabel(cs, fmt='%d')
    
        # Plot 500-hPa wind barbs in knots, regrid to reduce number of barbs
    
        # Make some nice titles for the plot (one right, one left)
        plt.title('500-hPa NAM Geopotential Heights (m), Wind Speed (kt), '+setting+' '+model, fontsize=8, loc='left')
        plt.title(dt.strftime('%m-%d-%Y %H:%M UTC'), loc='right')
    
        # Adjust image and show
        plt.subplots_adjust(bottom=0, top=1)
        plt.tight_layout()
        plt.savefig('../figures/Atmos/500_Heights_Winds_'+setting+'_'+file[11:-8]+'_'+model+'.png', dpi=200)
        plt.close()
    
        # Begin figure and set CONUS areal extent
        fig = plt.figure(1, figsize=(10, 8))
        ax = plt.subplot(111, projection=mapcrs)
        ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())

        # Add coastlines and state boundaries
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
        ax.add_feature(cfeature.STATES.with_scale('50m'))

        # Plot Colorfill of 700-hPa relative humidity -
        # normalize color to not have too dark of green at the top end
        clevs_700_relh = np.arange(70, 101, 2)
        cf = ax.contourf(lons, lats, relh_700, clevs_700_relh, cmap=plt.cm.Greens,
                         norm=plt.Normalize(70, 110), transform=datacrs)
        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
        cb.set_label('Rel. Humidity (%)')

        # Plot contours of 700-hPa geopotential height in black, solid lines
        clevs_700_hght = np.arange(0, 8000, 30)
        cs2 = ax.contour(lons, lats, hght_700, clevs_700_hght, colors='black', transform=datacrs)
        ax.clabel(cs2, fmt='%d', fontsize='large')

        # Add some useful titles
        plt.title('700-hPa NAM Geopotential Heights (m), Rel. Hum (%), '+setting+' '+model, fontsize=8, loc='left')
        plt.title(dt.strftime('%m-%d-%Y %H:%M UTC'), loc='right')

        plt.savefig('../figures/Atmos/700_Heights_RH_'+setting+'_'+file[11:-8]+'_'+model+'.png', dpi=200)
        plt.close()
    
    if plot_sfc == True:
    
        # Set up the projection that will be used for plotting
        mapcrs = ccrs.LambertConformal(central_longitude=-100,
                                       central_latitude=35,
                                       standard_parallels=(30, 60))
    
        # Set up the projection of the data; if lat/lon then PlateCarree is what you want
        datacrs = ccrs.PlateCarree()
    
        # 500 
        # Start the figure and create plot axes with proper projection
        fig = plt.figure(1, figsize=(10, 8))
        ax = plt.subplot(111, projection=mapcrs)
        ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    
        # Add geopolitical boundaries for map reference
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
        ax.add_feature(cfeature.STATES.with_scale('50m'))
    
        # Plot surface temperature in degrees Farenheit
        clevs_temp = np.arange(10, 120, 5)
        cf = ax.contourf(lons, lats, tmpk.to('degF'), clevs_temp, cmap=plt.cm.gist_rainbow_r,
                         transform=datacrs)
        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
        cb.set_label('Temperature (F)')
    
        #Plot MSLP in hPa
        # Plot MSLP
        clevmslp = np.arange(950, 1030., 4)
        cs2 = ax.contour(lons, lats, mslp.to('hPa'), clevmslp, colors='k', linewidths=1.25,
                         linestyles='solid', transform=datacrs)
        ax.clabel(cs2, fmt='%d')
    
        # Plot 500-hPa wind barbs in knots, regrid to reduce number of barbs
    
        # Make some nice titles for the plot (one right, one left)
        plt.title('Surface Pressure (hPa), Surface Temperature (degF) '+setting+' '+model, fontsize=8, loc='left')
        plt.title(dt.strftime('%m-%d-%Y %H:%M UTC'), fontsize=8, loc='right')
    
        # Adjust image and show
        plt.subplots_adjust(bottom=0, top=1)
        plt.tight_layout()
        plt.savefig('../figures/Surface/Sfc_Pres_Temp_'+setting+'_'+file[11:-8]+'_'+model+'.png', dpi=200)
        plt.close()
        
    if plot_soil == True:
        
        # Set up the projection that will be used for plotting
        mapcrs = ccrs.LambertConformal(central_longitude=-100,
                                       central_latitude=35,
                                       standard_parallels=(30, 60))
    
        # Set up the projection of the data; if lat/lon then PlateCarree is what you want
        datacrs = ccrs.PlateCarree()
    
        # 500 
        # Start the figure and create plot axes with proper projection
        fig = plt.figure(1, figsize=(10, 8))
        ax = plt.subplot(111, projection=mapcrs)
        ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    
        # Add geopolitical boundaries for map reference
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
        ax.add_feature(cfeature.STATES.with_scale('50m'))
    
        # Plot surface temperature in degrees Farenheit
        clevs_temp = np.arange(10, 120, 5)
        cf = ax.contourf(lons, lats, soil_t.to('degF'), clevs_temp, cmap=plt.cm.gist_rainbow_r,
                         transform=datacrs)

        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
        cb.set_label('Temperature (F)')
    
        # Plot 500-hPa wind barbs in knots, regrid to reduce number of barbs
    
        # Make some nice titles for the plot (one right, one left)
        plt.title('Soil Temperature (F) '+setting+' '+model, loc='left')
        plt.title(dt.strftime('%m-%d-%Y %H:%M UTC'), loc='right')
    
        # Adjust image and show
        plt.subplots_adjust(bottom=0, top=1)
        plt.tight_layout()
        plt.savefig('../figures/Soil/Soil_Temp_'+setting+'_'+file[11:-8]+'_'+model+'.png', dpi=200)
        plt.close()
    
    
    # Provide an update on the progress
    print('Finish Writing ', model)

### Loop through the 3 models that were used in this study
- MIROC
- GFDL
- NCAR CCSM

In [10]:
def mod_files(setting, plot_3d=False, plot_sfc=False, plot_soil=False,
              nam_filepath='/data/keeling/a/mgrover4/b/CMIP5_to_WRF/data/moore/',
              new_filepath='/data/keeling/a/mgrover4/b/CMIP5_to_WRF/modified_files/',
              models = ['MIROC', 'GFDL', 'NCAR'],):
    files = os.listdir(nam_filepath)
    for file in files:
        if 'nam' in file:
            # Loop through the different models
            for model in models:
    
                # Use the function defined in the previous cell
                modify_grib_file(model, nam_filepath, file, new_filepath, setting, plot_3d, plot_sfc, plot_soil)
        
        print(file)

In [11]:
mod_files('Control', plot_3d=True, plot_sfc=True, plot_soil=True)

Finish Writing  
Finish Writing  
Finish Writing  
namanl_218_20130521_0600_000.grb
Finish Writing  
Finish Writing  
Finish Writing  
namanl_218_20130521_0000_000.grb
Finish Writing  
Finish Writing  
Finish Writing  
namanl_218_20130520_1200_000.grb
Finish Writing  
Finish Writing  
Finish Writing  
namanl_218_20130520_1800_000.grb


In [12]:
mod_files('Atmos', plot_3d=True)

Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130521_0600_000.grb
Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130521_0000_000.grb
Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130520_1200_000.grb
Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130520_1800_000.grb


In [13]:
mod_files('Surface', plot_sfc=True)

Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130521_0600_000.grb
Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130521_0000_000.grb
Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130520_1200_000.grb
Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130520_1800_000.grb


In [14]:
mod_files('Soil', plot_soil=True)

Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130521_0600_000.grb
Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130521_0000_000.grb
Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130520_1200_000.grb
Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130520_1800_000.grb


In [15]:
mod_files('All')

Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130521_0600_000.grb
Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130521_0000_000.grb
Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130520_1200_000.grb
Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
namanl_218_20130520_1800_000.grb
